In [ ]:
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
import pyspark.sql.types as T
import csv
# import pandas as pd
# import numpy as np
import sys
from pyspark.sql import Window
from pyspark.sql.functions import rank, col
# import geohash2 as geohash
# import pygeohash as pgh
from functools import reduce
from pyspark.sql import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1707717459446_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
brq = spark.read.parquet('s3a://ada-prod-data/etl/data/brq/agg/agg_brq/monthly/BD/2023{10,11,12}/*.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
brq.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ifa: string (nullable = true)
 |-- device: struct (nullable = true)
 |    |-- device_vendor: string (nullable = true)
 |    |-- device_name: string (nullable = true)
 |    |-- device_manufacturer: string (nullable = true)
 |    |-- device_model: string (nullable = true)
 |    |-- device_year_of_release: string (nullable = true)
 |    |-- platform: string (nullable = true)
 |    |-- major_os: string (nullable = true)
 |    |-- device_category: string (nullable = true)
 |-- connection: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- mm_con_type_desc: string (nullable = true)
 |    |    |-- mm_carrier_name: string (nullable = true)
 |    |    |-- req_con_type: integer (nullable = true)
 |    |    |-- req_carrier_mnc: string (nullable = true)
 |    |    |-- req_carrier_name: string (nullable = true)
 |    |    |-- req_carrier_code: string (nullable = true)
 |    |    |-- req_carrier_mcc: string (nullable = true)
 |    |    |-- brq_count: lon

In [ ]:
brq2 = brq.select('ifa', F.explode('gps')).select('ifa', 'col.*')
gps_df=brq2.withColumn("day", date_format('last_seen', 'dd'))
gps_df=gps_df.withColumn("month", date_format('last_seen', 'MM'))
gps_df=gps_df.drop(gps_df.last_seen)
gps_df=gps_df.withColumn("new_geohash", substring(col("geohash"),1,7))
gps_df=gps_df.drop(gps_df.geohash)
gps_df=gps_df.withColumnRenamed("new_geohash","geohash")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
gps_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-------------------+---------+----------+-------------------+--------+--------+-------+-----+---+-----+-------+
|                 ifa|brq_count|         first_seen|longitude|state_name|               city|   state|latitude|country|ndays|day|month|geohash|
+--------------------+---------+-------------------+---------+----------+-------------------+--------+--------+-------+-----+---+-----+-------+
|0000b54a-cf1e-4ec...|        4|2023-12-01 03:49:41|  90.3657|       NaN|            Azimpur|   Dhaka| 23.7731|     BD|    2| 11|   12|wh0r0rf|
|0000b54a-cf1e-4ec...|        1|2023-12-10 08:23:31|   90.382|       NaN|            Azimpur|   Dhaka|  23.746|     BD|    1| 10|   12|wh0r0dy|
|0000b54a-cf1e-4ec...|       11|2023-12-09 03:13:45|   90.404|       NaN|             Paltan|   Dhaka| 23.7571|     BD|    1| 09|   12|wh0r1hy|
|0000bd4d-0ca9-446...|       18|2023-12-01 09:47:15|   89.566|       NaN|             Khulna|  Khulna| 22.8159|     BD|    3| 11|   12|t

In [ ]:
# br= spark.read.parquet('s3a://ada-prod-data/etl/data/brq/agg/agg_brq/monthly/BD/2024{01}/*.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# brq2 = br.select('ifa', F.explode('gps')).select('ifa', 'col.*')
# gps_df2=brq2.withColumn("day", date_format('last_seen', 'dd'))
# gps_df=gps_df2.withColumn("month", date_format('last_seen', 'MM'))
# gps_df2=gps_df2.drop(gps_df.last_seen)
# gps_df2=gps_df2.withColumn("new_geohash", substring(col("geohash"),1,6))
# gps_df2=gps_df2.drop(gps_df.geohash)
# gps_df2=gps_df2.withColumnRenamed("new_geohash","geohash")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
br=spark.read.csv('s3a://ada-dev/BD-DataScience/muntasir/barikoi/uttara.csv',header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
br.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geohash: string (nullable = true)

In [ ]:
join=br.join(gps_df,'geohash','inner')
# join=join.select('ifa')
# join=join.distinct()
# join.count()

join.printSchema()

join.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geohash: string (nullable = true)
 |-- ifa: string (nullable = true)
 |-- brq_count: long (nullable = true)
 |-- first_seen: timestamp (nullable = true)
 |-- longitude: float (nullable = true)
 |-- state_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- country: string (nullable = true)
 |-- ndays: long (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)

+-------+--------------------+---------+-------------------+---------+----------+-----+-----+--------+-------+-----+---+-----+
|geohash|                 ifa|brq_count|         first_seen|longitude|state_name| city|state|latitude|country|ndays|day|month|
+-------+--------------------+---------+-------------------+---------+----------+-----+-----+--------+-------+-----+---+-----+
|wh0rbbv|000563e4-1735-43c...|        3|2023-12-06 08:18:22|  90.3927|       NaN|Tungi|Dhaka| 23.8672|     BD|    2

In [ ]:
# Using max() function
join.select(max("brq_count")).show(truncate=False)
# Using min() function
join.select(min("brq_count")).show(truncate=False)
# Using mean() function
join.select(mean("brq_count")).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+
|max(brq_count)|
+--------------+
|168893        |
+--------------+

+--------------+
|min(brq_count)|
+--------------+
|1             |
+--------------+

+-----------------+
|avg(brq_count)   |
+-----------------+
|37.73024815731787|
+-----------------+

In [ ]:
# from pyspark.sql.functions import when

# # Define conditions and values for the new column
# join = join.withColumn("Data Usage", 
#                     when(join["brq_count"] <= 20, "LOW")
#                     .when((join["brq_count"] >= 21) & (join["brq_count"] <= 70), "MID")
#                     .when((join["brq_count"] >= 71) & (join["brq_count"] <= 150), "HIGH")
#                     .when(join["brq_count"] >= 151, "ULTRA HIGH"))



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
data=join.groupBy("geohash").agg(F.sum('brq_count').alias('brq_count_sum')).sort('brq_count_sum', ascending=False)
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
|geohash|brq_count_sum|
+-------+-------------+
|wh0rbbv|    707621904|
|wh0rc17|      1075302|
|wh0r9pe|       418352|
|wh0rc0d|       119849|
|wh0rc4h|        49661|
|wh0r9pc|        38769|
|wh0rc30|        29620|
|wh0r9pb|        28891|
|wh0rc0v|        23736|
|wh0rc18|        22421|
|wh0rc06|        18999|
|wh0rbcz|        18035|
|wh0r9pf|        17765|
|wh0rc0s|        17010|
|wh0rbfp|        16265|
|wh0rc45|        16094|
|wh0rc07|        15113|
|wh0rc15|        14053|
|wh0rbbh|        13559|
|wh0rbfh|        11357|
+-------+-------------+
only showing top 20 rows

In [ ]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- geohash: string (nullable = true)
 |-- brq_count_sum: long (nullable = true)

In [ ]:
cnt=join.groupBy("geohash").agg(F.countDistinct('ifa').alias('count')).sort('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
cnt.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
|geohash|   count|
+-------+--------+
|wh0rbbv|13154108|
|wh0rc17|   30301|
|wh0r9pe|    8991|
|wh0rc4h|    1134|
|wh0rc45|    1075|
|wh0rc15|     979|
|wh0rc1e|     810|
|wh0rc07|     760|
|wh0rc06|     658|
|wh0rbfh|     616|
|wh0rc0g|     599|
|wh0rc47|     498|
|wh0rc1g|     492|
|wh0r9pg|     487|
|wh0rc0e|     456|
|wh0r9pf|     431|
|wh0rc40|     395|
|wh0rc0f|     385|
|wh0rbfj|     385|
|wh0rc42|     370|
+-------+--------+
only showing top 20 rows

In [ ]:
colab=cnt.join(data,'geohash','inner')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
colab.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+-------------+
|geohash|   count|brq_count_sum|
+-------+--------+-------------+
|wh0rc02|     214|         7486|
|wh0rc1h|     257|         5237|
|wh0rc40|     395|         5563|
|wh0rc1z|     184|         4243|
|wh0rc45|    1075|        16094|
|wh0rc0e|     456|        10432|
|wh0rc0n|     149|         7205|
|wh0rbbv|13154108|    707621904|
|wh0rc0v|     162|        23736|
|wh0rc4h|    1134|        49661|
|wh0rc10|     146|         2825|
|wh0rc20|     266|         5492|
|wh0rbcm|     223|         5183|
|wh0rc1f|     139|         1623|
|wh0rc1n|     200|         4262|
|wh0rc0k|     157|         3237|
|wh0rbbr|     163|         5047|
|wh0rbbq|     166|         2567|
|wh0rc30|     213|        29620|
|wh0rbbs|     142|         2986|
+-------+--------+-------------+
only showing top 20 rows

In [ ]:
colab.coalesce(1).write.csv('s3a://ada-dev/BD-DataScience/muntasir/barikoi2/', mode='overwrite', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…